In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

os.environ['XILINX_VITIS'] = '/tools/Xilinx/Vitis/2024.2'
os.environ['PATH'] = '/tools/Xilinx/Vivado/2020.1/bin:' + os.environ['PATH']
os.environ['PATH'] = '/tools/Xilinx/Vitis_HLS/2024.2/bin:' + os.environ['PATH']

2025-04-09 15:47:37.865351: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-09 15:47:37.866874: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-09 15:47:37.885087: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-09 15:47:37.885100: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-09 15:47:37.885770: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load dataset
iris = load_iris()
X = iris.data
y = iris.target

# Optionally scale features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


In [3]:
# Build a simple neural network
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(4,)),      # 4 features in Iris
    tf.keras.layers.Dense(10, activation='relu', ),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 output classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])



In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                50        
                                                                 
 dense_1 (Dense)             (None, 8)                 88        
                                                                 
 dense_2 (Dense)             (None, 3)                 27        
                                                                 
Total params: 165 (660.00 Byte)
Trainable params: 165 (660.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
# Train the model
model.fit(x_train, y_train, epochs=50, batch_size=4, validation_split=0.1, verbose=1)

# Evaluate on test set
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"\nTest Accuracy: {accuracy * 100:.2f}%")

Epoch 1/50
27/27 [==============================] - 0s 3ms/step - loss: 1.4437 - accuracy: 0.3519 - val_loss: 1.3983 - val_accuracy: 0.1667
Epoch 2/50
27/27 [==============================] - 0s 798us/step - loss: 1.3140 - accuracy: 0.3519 - val_loss: 1.2944 - val_accuracy: 0.1667
Epoch 3/50
27/27 [==============================] - 0s 794us/step - loss: 1.2210 - accuracy: 0.3519 - val_loss: 1.2065 - val_accuracy: 0.1667
Epoch 4/50
27/27 [==============================] - 0s 783us/step - loss: 1.1371 - accuracy: 0.3611 - val_loss: 1.1318 - val_accuracy: 0.1667
Epoch 5/50
27/27 [==============================] - 0s 715us/step - loss: 1.0564 - accuracy: 0.3611 - val_loss: 1.0657 - val_accuracy: 0.1667
Epoch 6/50
27/27 [==============================] - 0s 729us/step - loss: 0.9719 - accuracy: 0.3796 - val_loss: 1.0028 - val_accuracy: 0.2500
Epoch 7/50
27/27 [==============================] - 0s 810us/step - loss: 0.8902 - accuracy: 0.5093 - val_loss: 0.9421 - val_accuracy: 0.4167
Epoch 8/

In [ ]:
import hls4ml 

config = hls4ml.utils.config_from_keras_model(model, granularity='model', backend='Vitis')

config['Model']['ReuseFactor'] = 64
hls_model = hls4ml.converters.convert_from_keras_model(model, 
                                                       hls_config=config, 
                                                       backend='VivadoAccelerator', 
                                                       output_dir='Projects/Iris', 
                                                       part='xc7z020clg400-1')
hls_model.compile()

Interpreting Sequential
Topology:
Layer name: input_1, layer type: InputLayer, input shapes: [[None, 4]], output shape: [None, 4]
Layer name: dense, layer type: Dense, input shapes: [[None, 4]], output shape: [None, 10]
Layer name: dense_1, layer type: Dense, input shapes: [[None, 10]], output shape: [None, 8]
Layer name: dense_2, layer type: Dense, input shapes: [[None, 8]], output shape: [None, 3]
Interpreting Sequential
Topology:
Layer name: input_1, layer type: InputLayer, input shapes: [[None, 4]], output shape: [None, 4]
Layer name: dense, layer type: Dense, input shapes: [[None, 4]], output shape: [None, 10]
Layer name: dense_1, layer type: Dense, input shapes: [[None, 10]], output shape: [None, 8]
Layer name: dense_2, layer type: Dense, input shapes: [[None, 8]], output shape: [None, 3]
Creating HLS model
Writing HLS project
Done


In [7]:

hls_model.build(csim=False, export=True, bitfile=True)

TypeError: VivadoBackend.build() got an unexpected keyword argument 'bitfile'

In [ ]:
!rm -rf Projects/Iris